# Python Data Structures for Geospatial Data Processing

### Exercise: 

1. Read the data contained in the file "some_data.geojson" into Python **using at least 2 different ways (packages)**. 
2. What kind of data does it include? 
    - What do the features represent? 
    - What geometry type? 
    - What coordinate reference system? 
    
**Note:** If you need a package which is not installed on mybinder, you can install it by opening a terminal on mybinder and executing `pip install some-package`.

In [ ]:
import sys, os

file_path = "./data/some_data.geojson"

In [ ]:
#geopands

import geopandas as gpd

try:
    fileobj = gpd.read_file(file_path)
    print(fileobj.head())
except Exception as err:
    print('Unable to open the file because of: ' + str(err) '\nPlease check your input file.')
    #sys.exit()
    
fileobj.geometry.crs
fileobj.crs
fileobj.geom_type

In [ ]:
# json

import json

with open(file_path) as src:
    data_json = json.load(src)
    
foo = data_json["features"][0]["geometry"]["coordinates"][0]#[0]
print(type(foo), len(foo))
foo[0]

In [ ]:
# osgeo - ogr

from osgeo import ogr
driver = ogr.GetDriverByName('GeoJSON') 
ds = driver.Open(file_path, 0) #read

print('"%s"'%ds.GetLayer().GetSpatialRef())

layer = ds.GetLayer()
schema = []
ldefn = layer.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    schema.append(fdefn.name)
print(schema)

feature = layer.GetNextFeature()
geometry = feature.GetGeometryRef()
print(geometry.GetGeometryName())

In [ ]:
# gdal

from osgeo import gdal
from osgeo import osr

ds = gdal.OpenEx(file_path)
layer = ds.GetLayer()

# get and print the spatial Reference system
spatialRef = layer.GetSpatialRef()
print(spatialRef)

# print the geometries
for feature in layer:
    geom = feature.GetGeometryRef()
    print(geom)

In [ ]:
# fiona

import fiona

source = fiona.open(file_path)
features = list(source)
crs = source.crs
metadata = source.meta
source.close()

with fiona.open(file_path) as source:
    feautres = list(source)
    crs = source.crs
    metadata = source.meta

### Why are there so many packages? 

In discussions about Python you will often read statements like this:

![gdal_vs_rasterio](img/gdal_vs_rasterio.png)

[Source: GitHub Issue - rasterio vs python gdal #11](https://github.com/inbo/niche_vlaanderen/issues/11)

### After completing this notebook, you will be able to...
1. write a class in Python. 
2. name the advantages and disadvantages of using ogr, shapely or your own implementation.
3. explain what _[Pythonic](https://docs.python-guide.org/writing/style/)_ means.

## Comparing ogr, shapely and pure Python

We will compare three different classes to **represent a polygon** in Python and how to **calculate its bounding box.**
1. The **`Geometry` class provided by `ogr` package**.
2. The **`Polygon` class provided by the `shapely` package.**
3. A simplified **Python implementation of a `Polygon` class** written by yourself.

#### The three classes should be compared in regard to:
* Execution time
* Programming time (how long did it take to write the code)
* Readability
* Flexibility (how easily can you adapt it to your needs)

In [ ]:
test_coordinates = [(0,1.5),(0.5,1),(1,1.5),(0.5,2),(0,1.5)]

### 1. OGR package

__E 1.1:__ Use the `ogr` package to create a polygon geometry object using the same `test_coordinates` as above.

In [ ]:
from osgeo import ogr

def create_polygon(coords):          
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for coord in coords:
        ring.AddPoint(coord[0], coord[1])

    # Create polygon
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    return poly

poly = create_polygon(test_coordinates)

__E 1.2:__ Calculate the envelope of the polygon. 

In [ ]:
extent = poly.GetEnvelope()
extent

### 2. Shapely package

__E 2.1:__ Use `shapely` package to create a polygon object using the same `test_coordinates` as above. Use Google to find out how to do this.

In [ ]:
from shapely import geometry

poly2 = geometry.Polygon(test_coordinates)
poly2

__E 2.2:__ Calculate the bounding box of the shapely polygon. Is the result the same as above?

In [ ]:
# ogr
poly2.bounds

In [ ]:
type(poly2.bounds)

In [ ]:
# Shapely
poly2.envelope

In [ ]:
type(poly2.envelope)

### Bonus:  Create a polyon using the `pygeos` package

### 3. Do-It-Yourself `Polygon` class

### Theory: Classes and objects in Python

In [ ]:
class Person:
    age = 5
    name = "Tom"
    
    def is_adult(self):
        if self.age >= 18:
            return True
        else:
            return False

In [ ]:
person_a = Person()
person_a.is_adult()

In [ ]:
class Person:
    name = None
    age = None

    def __init__(self, name, age):
        self.name = name
        self.age = age

In [ ]:
person_b = Person(name="Tom", age=25)

In [ ]:
# person_a = Person()

In [ ]:
# person_b.ist_adult()

__E 3.1:__ Write a class called `MyPolygon`. It should be created by passing a list of coordinates representing the nodes of the polygon, e.g. `MyPolygon(coordinates=[(1,1),(1,2),(2,2),(2,1),(1,1))])`. Calling `polygon.coordinates` should return the coordinates.

In [ ]:
class MyPolygon:
    coordinates = None

    def __init__(self, coordinates):
        self.coordinates = coordinates

In [ ]:
my_poly = MyPolygon(coordinates = test_coordinates)

In [ ]:
my_poly.coordinates

Test whether the instance of class `Polygon` returns the right value. If there is no error message, everything's fine. 

In [ ]:
assert my_poly.coordinates == test_coordinates, "Something's wrong"

__E 3.2:__ Add a method called `envelope` to the class which calculates the bounding box of the polygon. The method should return the bounding box as a list containing the minimum and maximum coordinates of the bounding box, i.e.`[minimum_x, minimum_y, maximum_x, maximum_y]`.

__Note:__ A bounding box is often reffered to as the envelope of a geometry.

Check if your results are the same as above.

In [ ]:
import numpy as np

class MyPolygon:
    coordinates = None

    def __init__(self, coordinates):
        self.coordinates = coordinates

    def envelope(self):
        xs = np.array(self.coordinates)[:, 0]
        ys = np.array(self.coordinates)[:, 1]
        return [min(xs), min(ys), max(xs), max(ys)]

In [ ]:
my_poly = MyPolygon(coordinates = test_coordinates)
my_poly.envelope()

## 4. Comparison of execution times
Compare the exectution times of the ogr, shapely and your own class using the magic command `%%timeit`. This function will execute the cell multiple times to get a good estimate of the execution time. 

__E 4.1:__ Compare the exection times of the calculation of the bounding box of all three classes. 

In [ ]:
%%timeit
# DIY class
my_poly.envelope()

In [ ]:
%%timeit
# ogr
poly2.bounds

In [ ]:
%%timeit
# shapely
poly2.envelope

<i><strong>Shapely - DIY - OGR</strong></i>

__Question:__ What do you observe when you compre the execution times of all three methods? Can you explain the difference in execution times?

#### Comparison bounding box creation and object creation

When choosing the most efficient way to calculate something, we also need to consider the _overhead_ of the calculation. The _overhead_ contains all the processing steps that need to be taken as a preparation before the execution of the desired calculation. Depending on the implementation, this can change your decision.  

__E 4.2:__ Measure the execution times of all three implementations __including the object creation.__ 

In [ ]:
%%timeit
# DIY class
my_poly = MyPolygon(test_coordinates)
my_poly.envelope()

In [ ]:
%%timeit
# ogr
poly = create_polygon(test_coordinates)
poly2.bounds

In [ ]:
%%timeit
# shapely
poly2 = geometry.Polygon(test_coordinates)
poly2.envelope

<i><strong>DIY - Shapely - OGR</strong></i>

__Question:__ What do you observe when you compare the execution times and the object creation of all three polygon implementations? Can you explain the difference in execution times?

__Your Answer:__

### 5. Questions for Discussion

__Q1:__ Based on your results of the exercises, evaluate the three methods in the table below. (Double click the cell, to edit it)


|            | Execution time <br> (fast - slow)| Programming time <br> (fast - slow)| Readability <br> (high - low) | Flexibility  <br> (high - low)       |
|------------|--------------|------------------|-------------|-------------------|
| __DIY Python__ | mid   | mid-slow      | mid     | high         |
| __OGR__        | slow     | slow-mid     | low     | mid         |
| __Shapely__    | fast     | fast         | high     | low      |

__Q2:__ What are the advantages and disadvantages of using ogr, shapely or your own implementation for vector data processing? 

__Your answers:__

### OGR and GDAL Python bindings
* `ogr` and `gdal` are automatically generated Python bindings (using SWIG) to the C libraries GDAL and OGR 
* So when you use `ogr` and `gdal` classes in Python you are actually executing C code. 
* Still, the syntax or `ogr` and `gdal` doesn't really feel like Python (e.g. it needs many lines of code) 
* `shapely` is a native Python package and is therefore closer to the way Python code is supposed to be written

## References

If you would like to take a deeper look at object oriented programming in Python take a look at the following resources: 

* [Object Oriented Programming in Python](https://github.com/TheDigitalCatOnline/thedigitalcatonline.github.com/tree/master/notebooks)
* [Python Tutorial](https://docs.python.org/3/tutorial/classes.html)
* [Abstract Classes](https://docs.python.org/3/library/abc.html#module-abc)
* [Computational Geometry in Python: From Theory to Application](https://www.toptal.com/python/computational-geometry-in-python-from-theory-to-implementation
)

    
